In [46]:
# Import necessary libraries
import pandas as pd
import numpy as np
import datetime
import re
import requests  # In a real scenario, you would use this to call external APIs
import time
from io import BytesIO


In [2]:
# =============================================================================
# PART 1: Address Enrichment for Orders
# =============================================================================

In [3]:
orders_df = pd.read_csv('orders_file.csv')

In [4]:
orders_df.head()

,order_number,date,city,state,Zip,$ sale
0,ch_3QcGwwD8UtMUbTdk0mC0Dj8x,2025-01-01,NaN,WI,54956,$67.00
1,ch_3QcGwND8UtMUbTdk1b3ZqCE3,2025-01-01,NaN,WI,54956,$14.97
2,ch_3QcGvVD8UtMUbTdk07xs732y,2025-01-01,NaN,WI,54956,$19.97
3,ch_3QcGumD8UtMUbTdk1bWLVF4q,2025-01-01,NaN,WI,54956,$74.00
4,ch_3QcEvAD8UtMUbTdk0ZHANuNZ,2024-12-31,NaN,NC,27028,$19.97


In [5]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95672 entries, 0 to 95671
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_number  95672 non-null  object 
 1   date          95672 non-null  object 
 2   city          0 non-null      float64
 3   state         69931 non-null  object 
 4   Zip           94282 non-null  object 
 5   $ sale        95672 non-null  object 
dtypes: float64(1), object(5)
memory usage: 4.4+ MB


In [16]:
orders_df.drop_duplicates(subset='order_number', inplace=True)

In [17]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95653 entries, 0 to 95671
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_number  95653 non-null  object 
 1   date          95653 non-null  object 
 2   city          0 non-null      float64
 3   state         69922 non-null  object 
 4   Zip           94263 non-null  object 
 5   $ sale        95653 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.1+ MB


In [33]:
# Function 1: Load Orders Data
def load_orders_data(filename):
    # Read the CSV file
    orders_df = pd.read_csv(filename)
    
    # Drop duplicate orders
    orders_df.drop_duplicates(subset='order_number', inplace=True)
    
    # Convert the 'date' column from string(object) format (e.g., "01/01/25") to datetime format.
    # Assuming the date format is MM/DD/YY, use appropriate format.
    orders_df['date'] = pd.to_datetime(orders_df['date'], format='%Y-%m-%d', errors='coerce')
    
    # Convert sales from string (e.g., "$55.00") to a float.
    # Removing the dollar sign and convert to float.
    orders_df['sales'] = orders_df['$ sale'].replace('[\$,]', '', regex=True).astype(float)
    
    # Drop the original '$sales' column as we now have a numeric 'sales' column.
    orders_df.drop(columns=['$ sale'], inplace=True)
    
    return orders_df

In [34]:
order_df = load_orders_data('orders_file.csv')

In [35]:
##validating

In [36]:
order_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95653 entries, 0 to 95671
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_number  95653 non-null  object        
 1   date          95653 non-null  datetime64[ns]
 2   city          0 non-null      float64       
 3   state         69922 non-null  object        
 4   Zip           94263 non-null  object        
 5   sales         95653 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 5.1+ MB


In [37]:
##function is correct 

In [38]:
# Function 2: Load and Process IP Addresses

def load_and_merge_ip_data(order_df , ip_file):
    """
    written for my understanding
    Now we use the order_df which has city, stae and zip code of some orders, rest of the orders have ip_addreses given,
    let's get the city, stae and the zip code for those orders wothout address using zipcode 
    """
    # using the already cleaned orders df from the function before
    orders_df = order_df

    # Load the IP addresses file
    ip_df = pd.read_csv(ip_file)

    # Merge orders with IP addresses using order_number
    merged_df = orders_df.merge(ip_df, on="order_number", how="left")

    # Select only required columns, leaving city cause ot's nulll completely
    result_df = merged_df[['order_number', 'state', 'Zip']]

    return result_df


ip_file = "ip_addresses.csv"
final_df = load_and_merge_ip_data(order_df, ip_file)


print(final_df.head())


                  order_number state    Zip
0  ch_3QcGwwD8UtMUbTdk0mC0Dj8x    WI  54956
1  ch_3QcGwND8UtMUbTdk1b3ZqCE3    WI  54956
2  ch_3QcGvVD8UtMUbTdk07xs732y    WI  54956
3  ch_3QcGumD8UtMUbTdk1bWLVF4q    WI  54956
4  ch_3QcEvAD8UtMUbTdk0ZHANuNZ    NC  27028


In [39]:
# Function 3. Update Orders and Export Data:  
def update_orders_with_location(final_df):
    ## just simple export becasue no SQL DB connection or API information is provided, dummy queries are listed below 
    final_df.to_excel('new_orders_report_with_location_info.xlsx', index = False)

In [40]:
# =============================================================================
# PART 2: Sales Report Generation
# =============================================================================

In [43]:
# Function 4. Custom Report Generation:
def generate_sales_report(order_df, state, year, output_file):
    """
    Develop a function that accepts state and year as inputs.  
    Aggregate sales by state, city, and quarter.  
    Generate an Excel file containing the relevant sales data.  (Use IL_state_sales_report_2021.xlsx as a reference. Output should be in the same format)
    Orders without a city value can be ignored.
    """
    # Filter orders by the given state and year, city is removed because it dosen't have any value
    filtered_df = order_df[(order_df['state'] == state) & 
                            (order_df['date'].dt.year == year)
                            ]
    
    # Creating a quarter column based on the month of the order.
    filtered_df['quarter'] = filtered_df['date'].dt.month.apply(lambda x: f"Q{((x-1)//3)+1}")
    
    # aggregate by state and quarter and getting the total sales
    report_df = filtered_df.groupby(['state', 'quarter'])['sales'].sum().reset_index()
    
    # Pivot the table to have quarters as columns if needed (to match the reference format)
    report_pivot = report_df.pivot_table(index=['state'], columns='quarter', values='sales', fill_value=0).reset_index()
   
    
    # Export the report to an Excel file
    report_pivot.to_excel(output_file, index=False)
    print(f"Sales report generated and saved to {output_file}")

In [44]:
generate_sales_report(order_df, 'IL', 2021, 'output_file_flodata.xlsx')

Sales report generated and saved to output_file_flodata.xlsx


In [47]:
# =============================================================================
# SQL STATEMENTS (as comments)
# =============================================================================
"""
-- SQL for creating necessary tables:

-- Orders table
CREATE TABLE orders (
    order_number VARCHAR(50) PRIMARY KEY,
    date DATE,
    city VARCHAR(100),
    state VARCHAR(2),
    zip VARCHAR(10),
    sales DECIMAL(10,2)
);

-- IP addresses table (to store raw IP data linked to orders)
CREATE TABLE ip_addresses (
    order_number VARCHAR(50),
    ip_address VARCHAR(50),
    PRIMARY KEY (order_number, ip_address)
);

-- IP location cache table to avoid redundant API calls
CREATE TABLE ip_location_cache (
    ip_address VARCHAR(50) PRIMARY KEY,
    city VARCHAR(100),
    state VARCHAR(2),
    zip VARCHAR(10)
);

-- SQL for inserting/updating orders (example for PostgreSQL):
INSERT INTO orders (order_number, date, city, state, zip, sales)
VALUES ('12345', '2021-01-01', 'Milwaukee', 'WI', '53202', 55.00)


-- SQL for updating orders with location details:
UPDATE orders
SET city = ip_loc.city,
    state = ip_loc.state,
    zip = ip_loc.zip
FROM ip_location_cache ip_loc
WHERE orders.order_number IN (
    SELECT order_number FROM ip_addresses WHERE ip_address = ip_loc.ip_address
);
"""

"\n-- SQL for creating necessary tables:\n\n-- Orders table\nCREATE TABLE orders (\n    order_number VARCHAR(50) PRIMARY KEY,\n    date DATE,\n    city VARCHAR(100),\n    state VARCHAR(2),\n    zip VARCHAR(10),\n    sales DECIMAL(10,2)\n);\n\n-- IP addresses table (to store raw IP data linked to orders)\nCREATE TABLE ip_addresses (\n    order_number VARCHAR(50),\n    ip_address VARCHAR(50),\n    PRIMARY KEY (order_number, ip_address)\n);\n\n-- IP location cache table to avoid redundant API calls\nCREATE TABLE ip_location_cache (\n    ip_address VARCHAR(50) PRIMARY KEY,\n    city VARCHAR(100),\n    state VARCHAR(2),\n    zip VARCHAR(10)\n);\n\n-- SQL for inserting/updating orders (example for PostgreSQL):\nINSERT INTO orders (order_number, date, city, state, zip, sales)\nVALUES ('12345', '2021-01-01', 'Milwaukee', 'WI', '53202', 55.00)\n\n\n-- SQL for updating orders with location details:\nUPDATE orders\nSET city = ip_loc.city,\n    state = ip_loc.state,\n    zip = ip_loc.zip\nFROM ip_